<a href="https://colab.research.google.com/github/adudars/PedidON/blob/master/arquivos/9.9_ConsultasSelfJoin%26View.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.9 CONSULTAS COM SELF JOIN E VIEW

## ESTABELECENDO CONEXÃO




### Instalando bibliotecas de acesso

In [1]:
#!pip install --user psycopg2-binary
#!pip install --user pandas

### Importando bibliotecas

In [3]:
import pandas as pd
import psycopg2

### Conectando com o banco através do psycopg2

In [23]:
conn = psycopg2.connect(host="lallah.db.elephantsql.com", database="nxthbnlu", user="nxthbnlu", password="buyAYKsY17pqexg1grIcJaWm0gHBsO2F")

## CONSULTA COM SELF JOIN

Uma vez que o SELF JOIN é utilizado para criar relacionamento entre informações de uma mesma tabela, visando portanto o relacionamento entre os registros pertencentes a mesma, e isso não ocorre em nosso banco de dados PedidON, substituímos, então, uma consulta com self join por uma criação de view, conforme orientado.

## VIEWS

### Funcionário destaque

In [5]:
cursor_view_destaque = conn.cursor()
cursor_view_destaque.execute("""create or replace view funcionario_destaque as
                        select nome_funcionario, count(Fk_id_Funcionario) as atendimentos
                        from comanda as c
                        inner join funcionario as f on 
                        (f.id_Funcionario = c.Fk_id_Funcionario)
                        where date_part('month',data) = date_part('month',(current_date + INTERVAL '-1 month')) and date_part('year',data) = date_part('year',(current_date + INTERVAL '-1 month'))
                        group by nome_funcionario
                        order by atendimentos desc
                        limit 1""")


In [6]:
destaque = pd.read_sql_query("""select * from funcionario_destaque""", conn)

In [7]:
destaque

,nome_funcionario,atendimentos
0,Bernardo,6


### Frequência clientes no mês anterior

In [8]:
cursor_view_frequencia = conn.cursor()
cursor_view_frequencia.execute("""create or replace view frequencia_clientes as
                        select nome_cliente, count(distinct data) as frequencia
                        from cliente as c
                        inner join comanda as cd on
                        (c.id_cliente = cd.fk_id_cliente)
                        where date_part('month',data) = date_part('month',(current_date + INTERVAL '-1 month')) and date_part('year',data) = date_part('year',(current_date + INTERVAL '-1 month'))
                        group by nome_cliente
                        order by frequencia desc;""")


In [9]:
frequencia = pd.read_sql_query("""select * from frequencia_clientes""", conn)

In [10]:
frequencia

,nome_cliente,frequencia
0,Jorge,4
1,Clarice,3
2,Lara,3
3,Davi,3
4,Mauro,3
5,Diego,3
6,Bento,3
7,Antonella,3
8,Isabella,3
9,Henrique,3


### Cardápio TOP 3 no mês anterior

In [11]:
cursor_view_top3 = conn.cursor()
cursor_view_top3.execute("""create or replace view cardapio_top_3 as
                            select ic.nome_item, ic.preco as valor, sum(quantidade) as total_pedidos
                            from pedido as p
                            inner join item_cardapio as ic on
                            (ic.id_itemcardapio = p.fk_id_itemcardapio)
                            inner join comanda as cd on
                            (p.fk_id_comanda = cd.id_comanda)
                            where date_part('month',data) = date_part('month',(current_date + INTERVAL '-1 month')) and date_part('year',data) = date_part('year',(current_date + INTERVAL '-1 month'))
                            group by ic.nome_item, ic.preco
                            order by total_pedidos desc
                            limit 3;""")


In [12]:
top3 = pd.read_sql_query("""select * from cardapio_top_3""", conn)

In [13]:
top3

,nome_item,valor,total_pedidos
0,Appia,25.9,6
1,Brisa,26.0,5
2,Cachaça,11.9,4


### Itens de cardápio

In [24]:
cursor_view_itens = conn.cursor()
cursor_view_itens.execute("""create or replace view Itens_cardapio as
                            select nome_item as nome, descricao_item as descricao, preco, nome_categoria as categoria, descricao_tamanho as tamanho
                            from item_cardapio as ic
                            inner join categoria as c on
                            (ic.fk_id_categoria = c.id_categoria)
                            inner join tamanho as t on
                            (ic.fk_id_tamanho = t.id_tamanho);""")
conn.commit()

In [25]:
itens = pd.read_sql_query("""select * from itens_cardapio""", conn)

In [26]:
itens

,nome,descricao,preco,categoria,tamanho
0,Rua Sete,"Filé Mignon ao conhaque, servido com pães da casa",55.00,entrada,1 pessoa
1,Reta da Penha,"Filé Mignon marinado na mostarda antiga, servi...",56.90,entrada,1 pessoa
2,Cidade Sol,"Whiskey Jack Daniels, maracujá, mix de cítrico...",29.90,drink,330ml
3,Barlavento,"Gin, xarope de morango, maracujá, suco de lara...",25.90,drink,330ml
4,Bons ventos te trazem,"Gin, redução de flor de sabugueiro, maracujá, ...",25.90,drink,330ml
5,Brasil Brasil,"Cachaça de Umburana, manjericão orgânico, mix ...",24.90,drink,2l
6,Pituã Island,"Absinto, xarope de pêssego, mix de cítricos, x...",27.90,drink,500ml
7,Mojito,"Rum Havana Club, Mix de cítricos, hortelã, xar...",31.90,drink,1l
8,Gruta da onça,"Entrecôte, salsa chermoula e legumes tostados",177.99,principal,3 pessoas
9,Paneleiras,Releitura da moqueca capixabas - Peixe grelhad...,217.99,principal,4 pessoas


### Faturamento do dia corrente

In [27]:
cursor_view_faturamento = conn.cursor()
cursor_view_faturamento.execute("""create or replace view faturamento_diario as
                                select sum(preco * quantidade) as valor_total_dia, to_char(data, 'dd-mm-yyyy')
                                from item_cardapio as cp
                                inner join pedido as p on
                                (p.fk_id_itemcardapio = cp.id_itemcardapio)
                                inner join comanda as cd on
                                (p.fk_id_comanda = cd.id_comanda)
                                where data = current_date
                                group by data;""")
conn.commit()

In [28]:
faturamento = pd.read_sql_query("""select * from faturamento_diario""", conn)

In [29]:
faturamento

,valor_total_dia,to_char


**OBS:** De fato não há faturamento no dia corrente, por isso a consulta veio vazia

### Gasto cliente no mês anterior

In [31]:
cursor_view_gasto = conn.cursor()
cursor_view_gasto.execute("""create or replace view gasto_cliente as
                              select nome_cliente, sum(quantidade * preco) as valor_comanda
                              from comanda as cd
                              inner join pedido as p on
                              (cd.id_comanda = p.fk_id_comanda)
                              inner join item_cardapio as cp on
                              (p.fk_id_itemcardapio = cp.id_itemcardapio)
                              inner join cliente as c on
                              (cd.fk_id_cliente = c.id_cliente)
                              where date_part('month',data) = date_part('month',(current_date + INTERVAL '-1 month')) and date_part('year',data) = date_part('year',(current_date + INTERVAL '-1 month'))
                              group by nome_cliente
                              order by valor_comanda desc;""")
conn.commit()


In [32]:
gasto = pd.read_sql_query("""select * from gasto_cliente""", conn)

In [33]:
gasto

,nome_cliente,valor_comanda
0,Clara,1929.60
1,Davi,1411.49
2,Henrique,1168.79
3,Ana,1036.98
4,Tais,906.87
5,Isabella,880.67
6,Bento,780.97
7,Moises,634.00
8,Luiza,560.99
9,Jorge,485.30
